# Warning
## This process is very very time consuming, use tf.data for faster data pipelines.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [ ]:
train = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/train.csv')
test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train['StudyInstanceUID'] = train['StudyInstanceUID'] + '.jpg'

In [ ]:
train.drop('PatientID',axis=1,inplace=True)

In [ ]:
train.info()

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input,
            validation_split=0.3,
            rotation_range=25,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            shear_range=0.15,
            zoom_range=0.1,
            fill_mode='nearest'
            )

In [ ]:
test_datagen = keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input)

In [ ]:
train_ds = train_datagen.flow_from_dataframe(
    dataframe=train, directory="../input/ranzcr-clip-catheter-line-classification/train", 
    x_col="StudyInstanceUID", y_col= test.columns[1:].to_list(),
    class_mode="raw", target_size=(313,313), batch_size=16, subset = "training")

valid_ds = train_datagen.flow_from_dataframe(
    dataframe=train, directory="../input/ranzcr-clip-catheter-line-classification/train", 
    x_col="StudyInstanceUID", y_col=test.columns[1:].to_list(),
    class_mode= "raw", target_size=(313,313), batch_size=16, subset = "validation")

In [ ]:
'''
def get_model():
    base_model = tf.keras.applications.InceptionResNetV2(include_top=False, weights='imagenet')
    base_model.trainable = True 
    inputs = keras.layers.Input(shape=(313,313,3))
    base_m = base_model(inputs)
    global_avg = keras.layers.GlobalAveragePooling2D()(base_m)
    batch_norm_1 = keras.layers.BatchNormalization()(global_avg)
    dense_1 = keras.layers.Dense(128, activation='relu')(batch_norm_1)
    dropout_1 = keras.layers.Dropout(0.4)(dense_1)
    batch_norm_2 = keras.layers.BatchNormalization()(dropout_1)
    dense_2 = keras.layers.Dense(128, activation='relu')(batch_norm_2)
    dropout_2 = keras.layers.Dropout(0.4)(dense_2)
    batch_norm_3 = keras.layers.BatchNormalization()(dropout_2)
    dense_3 = keras.layers.Dense(64, activation='relu')(batch_norm_3)
    dropout_3 = keras.layers.Dropout(rate=0.4)(dense_3)
    predictions = keras.layers.Dense(11, activation='sigmoid')(dropout_3)
    
    model = keras.Model(inputs=inputs, outputs=predictions)
    model.compile("adam", loss="binary_crossentropy", metrics=['acc'])
    
    return model
    
model = get_model()
model.summary()
'''

### First I trained 10 epochs with the above model and saved it. I again loaded it and trained. 

In [ ]:
model = keras.models.load_model('../input/imagedatagen/Best_Model.h5')

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint('./Best_Model.h5', 
                             monitor= 'val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             mode= 'min', 
                             save_weights_only = False)

In [ ]:
history = model.fit(train_ds, 
                    validation_data = valid_ds,
                    validation_steps = valid_ds.n//valid_ds.batch_size,
                    epochs = 20, 
                    callbacks = checkpoint,
                    steps_per_epoch = train_ds.n//train_ds.batch_size
                   )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()